In [8]:
import os

In [9]:
%pwd

'/Users/rupeshyadav/Documents/MLFLOW/End-To-End-MLOPs-Credit_Card_Fraud_Detection/research'

In [10]:
os.chdir("../")

In [11]:
%pwd

'/Users/rupeshyadav/Documents/MLFLOW/End-To-End-MLOPs-Credit_Card_Fraud_Detection'

Entity Definition

In [12]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

Configuration Manager

In [13]:
from Credit_Card_Fraud_Detection.constants import *
from Credit_Card_Fraud_Detection.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

Data Validation Strategy

In [15]:
import os
from Credit_Card_Fraud_Detection import logger

In [16]:
class DataValidationStrategy:
    """
    class for data validation strategy
    """

    def __init__(self, config: DataValidationConfig):
        self.config = config
        self.validation_report = {}
        self.data = pd.read_csv(self.config.unzip_data_dir)

    def validate_all_columns(self) -> bool:
        """
        validates all the columns of the data with the schema defined
        :return: validation status: True if all columns matches the schema of data
        """
        try:

            all_cols = list(self.data.columns)
            logger.info(all_cols)

            all_schema = self.config.all_schema.keys()
            print("=========values============")
            print(self.config.all_schema.values())
            
            for column, expected_type in self.config.all_schema.items():
                print(column,expected_type)
                # if self.data[column].dtype != expected_type:
                    
                    # self.validation_status = False
                    # self.validation_report.append(f"Invalid Data Type in {column}")
                
            print("=========values============")
            
            logger.info(all_schema)

            all_schema_set = set(all_schema)
            column_validation_status = True  # Set by default as True

            for col in all_cols:
                if col not in all_schema_set:
                    validation_status = False
                    break  # No need to continue checking if validation fails

            self.validation_report["column_validation_status"] = column_validation_status

            # # Write the final validation status
            # with open(self.config.STATUS_FILE, 'w') as f:
            #     f.write(f"Column Validation status: {column_validation_status}")

            return column_validation_status

        except Exception as e:
            raise e

    def check_missing_values(self) -> bool:
        missing_value_status = True # Initially assumed to be no missing values
        data = pd.read_csv(self.config.unzip_data_dir)

        missing_values = data.isnull().sum()
        if missing_values.any():
            missing_value_status = False # If the missing values is present

        self.validation_report["missing_value_status"] = missing_value_status
        
        return missing_value_status
        
            
    def check_data_types(self):
        
        data_type_validation_status = True
        for column, expected_type in self.config.all_schema.items():
            if self.data[column].dtype != expected_type:
                data_type_validation_status = False
                break


        self.validation_report["data_type_validation_status"] = data_type_validation_status

        # Write the final validation status
        with open(self.config.STATUS_FILE, 'w') as f:
             f.write(f"validation report: {self.validation_report}")

        return data_type_validation_status



         

         

        

In [17]:
import pandas as pd

In [18]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidationStrategy(config=data_validation_config)
    data_validation.validate_all_columns()
    data_validation.check_missing_values()
    data_validation.check_data_types()
except Exception as e:
    raise e

[2024-01-02 16:17:58,857: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-02 16:17:58,858: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-02 16:17:58,861: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-02 16:17:58,861: INFO: common: created directory at: artifacts]
[2024-01-02 16:17:58,862: INFO: common: created directory at: artifacts/data_validation]
[2024-01-02 16:17:59,607: INFO: 3948677739: ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class']]
=========values============
dict_values(['float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'fl